<a href="https://colab.research.google.com/github/netra-poonia/Incentive-Calculator-Colab-notebooks/blob/main/Relationship_Manager_Bonus_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## D-day code

In [50]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

class RMBonusCalculator:
    """
    Complete RM (Relationship Manager) Bonus Calculation System

    Features:
    1. Volume Payout (Slab-based like income tax)
    2. Effort Payout (Matrix-based on disbursals vs logins)
    3. Vintage-based structures (0-3M, 4-6M, 6M+)
    4. Milestone rewards (Silver/Gold/Platinum)
    5. Month evaluation logic based on joining date
    6. Dynamic evaluation period based on financial quarters
    """

    def __init__(self):
        self.setup_configuration()

    def setup_configuration(self):
        """Initialize all configuration data"""

        # Volume payout slabs for different vintage categories
        self.VOLUME_PAYOUT_SLABS = {
            'secured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 30, 'bonus_rate': 0.0020},
                {'threshold_multiplier': 50, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 80, 'bonus_rate': 0.004},
                {'threshold_multiplier': 120, 'bonus_rate': 0.005},
            ],
            'secured_direct': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.0012},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0018},
                {'threshold_multiplier': 30, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 60, 'bonus_rate': 0.0030},
            ],
            'unsecured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 5, 'bonus_rate': 0.004},
                {'threshold_multiplier': 15, 'bonus_rate': 0.007},
                {'threshold_multiplier': 30, 'bonus_rate': 0.01},
            ],
        }

        # BAU Effort Matrices (logins vs disbursals) - CORRECTED AXES
        self.BAU_SECURED_EFFORT_MATRIX = {
            (0, 0):            {(0, 0): 1000, (1, 1): 1200, (2, 2): 1500, (3, 3): 1800, (4, float('inf')): 2000},
            (1, 4):            {(0, 0): 1200, (1, 1): 1500, (2, 2): 1800, (3, 3): 2000, (4, float('inf')): 3000},
            (5, 6):            {(0, 0): 1500, (1, 1): 1800, (2, 2): 3000, (3, 3): 3500, (4, float('inf')): 4000},
            (7, float('inf')): {(0, 0): 2000, (1, 1): 2500, (2, 2): 3500, (3, 3): 4000, (4, float('inf')): 5000},
        }

        self.BAU_UNSECURED_EFFORT_MATRIX = {
            (0, 0):                {(0, 0): 0,   (1, 3): 300, (3, 4): 400, (5, 7): 500, (8, float('inf')): 750},
            (1, 9):                {(0, 0): 300, (1, 3): 400, (3, 4): 500, (5, 7): 750, (8, float('inf')): 850},
            (10, 13):              {(0, 0): 400, (1, 3): 500, (3, 4): 750, (5, 7): 850, (8, float('inf')): 1000},
            (14, float('inf')):    {(0, 0): 500, (1, 3): 750, (3, 4): 850, (5, 7): 1000, (8, float('inf')): 1200},
        }

        # Vintage 0-3 Months
        self.VINTAGE_VOLUME_SLABS_0_3M = {
            'secured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 5, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 20, 'bonus_rate': 0.0035},
                {'threshold_multiplier': 30, 'bonus_rate': 0.0045},
                {'threshold_multiplier': 45, 'bonus_rate': 0.005},
            ],
            'secured_direct': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.0012},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0016},
                {'threshold_multiplier': 25, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 40, 'bonus_rate': 0.0030},
            ],
            'unsecured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 3, 'bonus_rate': 0.004},
                {'threshold_multiplier': 6, 'bonus_rate': 0.008},
                {'threshold_multiplier': 9, 'bonus_rate': 0.01},
            ],
        }

        self.VINTAGE_SECURED_EFFORT_MATRIX_0_3M = {
            (0, 0): {(0, 0): 1500, (1, 1): 1800, (2, float('inf')): 2000},
            (1, 1): {(0, 0): 1800, (1, 1): 2000, (2, float('inf')): 2500},
            (2, 2): {(0, 0): 2500, (1, 1): 3000, (2, float('inf')): 3500},
            (3, float('inf')): {(0, 0): 3000, (1, 1): 3500, (2, float('inf')): 4000},
        }

        self.VINTAGE_UNSECURED_EFFORT_MATRIX_0_3M = {
            (0, 1): {(0, 0): 0, (1, 1): 200, (2, 2): 300, (3, 3): 400, (4, float('inf')): 650},
            (2, 3): {(0, 0): 200, (1, 1): 300, (2, 2): 400, (3, 3): 650, (4, float('inf')): 750},
            (4, 5): {(0, 0): 300, (1, 1): 400, (2, 2): 650, (3, 3): 750, (4, float('inf')): 900},
            (6, float('inf')): {(0, 0): 400, (1, 1): 650, (2, 2): 750, (3, 3): 900, (4, float('inf')): 1200},
        }

        # Vintage 4-6 Months
        self.VINTAGE_VOLUME_SLABS_4_6M = {
            'secured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0020},
                {'threshold_multiplier': 25, 'bonus_rate': 0.0030},
                {'threshold_multiplier': 40, 'bonus_rate': 0.0040},
                {'threshold_multiplier': 60, 'bonus_rate': 0.0045},
            ],
            'secured_direct': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.0010},
                {'threshold_multiplier': 10, 'bonus_rate': 0.0015},
                {'threshold_multiplier': 25, 'bonus_rate': 0.0025},
                {'threshold_multiplier': 40, 'bonus_rate': 0.0030},
            ],
            'unsecured': [
                {'threshold_multiplier': 0, 'bonus_rate': 0.000},
                {'threshold_multiplier': 4, 'bonus_rate': 0.004},
                {'threshold_multiplier': 12, 'bonus_rate': 0.008},
                {'threshold_multiplier': 16, 'bonus_rate': 0.01},
            ],
        }

        self.VINTAGE_SECURED_EFFORT_MATRIX_4_6M = {
            (0, 1): {(0, 0): 1200, (1, 1): 1500, (2, 2): 1800, (3, float('inf')): 2000},
            (2, 2): {(0, 0): 1500, (1, 1): 1800, (2, 2): 2000, (3, float('inf')): 2500},
            (3, 3): {(0, 0): 1800, (1, 1): 2000, (2, 2): 3000, (3, float('inf')): 3500},
            (4, float('inf')): {(0, 0): 2000, (1, 1): 2500, (2, 2): 3500, (3, float('inf')): 4000},
        }

        self.VINTAGE_UNSECURED_EFFORT_MATRIX_4_6M = {
            (0, 3): {(0, 0): 0, (1, 1): 200, (2, 2): 300, (3, 4): 400, (5, float('inf')): 500},
            (4, 5): {(0, 0): 200, (1, 1): 250, (2, 2): 350, (3, 4): 450, (5, float('inf')): 550},
            (6, 7): {(0, 0): 250, (1, 1): 350, (2, 2): 450, (3, 4): 550, (5, float('inf')): 650},
            (8, 9): {(0, 0): 350, (1, 1): 450, (2, 2): 550, (3, 4): 650, (5, float('inf')): 750},
            (10, float('inf')): {(0, 0): 450, (1, 1): 550, (2, 2): 650, (3, 4): 750, (5, float('inf')): 1000},
        }

        # Milestone Configuration
        self.MILESTONE_THRESHOLDS = {
            'silver': {'min_secured_files': 2, 'min_volume': 1500000},
            'gold': {'min_secured_files': 3, 'min_volume': 2000000},
            'platinum': {'min_secured_files': 4, 'min_volume': 3000000},
        }

        # Configurable evaluation periods based on financial quarters
        # Q1: Apr-Jun, Q2: Jul-Sep, Q3: Oct-Dec, Q4: Jan-Mar
        # Will be calculated dynamically based on joining date
        self.EVALUATION_PERIOD_MONTHS = None  # Will be set by calculate_evaluation_period()

        self.MILESTONE_QUALIFYING_MONTHS = {
            'silver': None,  # Will be calculated dynamically
            'gold': None,    # Will be calculated dynamically
            'platinum': None # Will be calculated dynamically
        }

        self.MILESTONE_SALARY_HIKE = {
            'silver': {'below_25k': 0.08, 'above_25k': 0.05},
            'gold': {'below_25k': 0.15, 'above_25k': 0.12},
            'platinum': {'below_25k': 0.20, 'above_25k': 0.15},
        }

        self.MILESTONE_SATURDAY_OFF = {
            'silver': 1,
            'gold': 2,
            'platinum': 3,
        }

    def calculate_evaluation_period(self, joining_date: datetime) -> int:
        """
        Calculate evaluation period based on financial quarter and joining date

        Financial Quarters:
        Q1: April 1st to June 30th
        Q2: July 1st to September 30th
        Q3: October 1st to December 31st
        Q4: January 1st to March 31st

        Logic if joining month is evaluated:
        - If joins in 1st month of quarter: 6 months evaluation (BAU)
        - If joins in 2nd month of quarter: 5 months evaluation
        - If joins in 3rd month of quarter: 7 months evaluation

        Logic if joining month not evaluated:
        - If joins in 1st month of quarter: 5 months evaluation
        - If joins in 2nd month of quarter: 7 months evaluation
        - If joins in 3rd month of quarter: 6 months evaluation (BAU)

        Returns:
            int: Number of months for evaluation (5, 6, or 7)
        """

        # Determine which month of the financial quarter
        month = joining_date.month
        day = joining_date.day

        # Check if joining month is evaluated (joined on or before the 10th)
        is_joining_month_evaluated = day <= 10

        # Map calendar months to financial quarter months
        if month in [4, 7, 10, 1]:  # 1st month of quarter
            return 6 if is_joining_month_evaluated else 5
        elif month in [5, 8, 11, 2]:  # 2nd month of quarter
            return 5 if is_joining_month_evaluated else 7
        elif month in [6, 9, 12, 3]:  # 3rd month of quarter
            return 7 if is_joining_month_evaluated else 6
        else:
            return 6  # Default BAU

    def update_milestone_qualifying_months(self, evaluation_period: int):
        """Update milestone qualifying months based on evaluation period"""
        if evaluation_period == 5:
            self.MILESTONE_QUALIFYING_MONTHS = {
                'silver': 3,
                'gold': 3,
                'platinum': 3,
            }
        elif evaluation_period == 6:
            self.MILESTONE_QUALIFYING_MONTHS = {
                'silver': 3,
                'gold': 4,
                'platinum': 4,
            }
        elif evaluation_period == 7:
            self.MILESTONE_QUALIFYING_MONTHS = {
                'silver': 4,
                'gold': 5,
                'platinum': 5,
            }
        else:
            # Default to BAU (6 months) if not matched
            self.MILESTONE_QUALIFYING_MONTHS = {
                'silver': 3,
                'gold': 4,
                'platinum': 4,
            }

    def is_joining_month_after_10th(self, joining_date: datetime, evaluation_date: datetime) -> bool:
        """
        Check if this is the joining month and joined after 10th
        """
        if joining_date.year == evaluation_date.year and joining_date.month+1 == evaluation_date.month:
            return joining_date.day > 10
        return True  #changed to true from false

    def get_vintage_category(self, joining_date: datetime, evaluation_date: datetime, is_joining_month_after_10th: bool) -> str:
        """
        Determine vintage category based on joining and evaluation dates,
        considering if the joining month counts towards vintage.
        """
        # Calculate months difference, subtracting 1 if joining month is not evaluated
        months_diff = (evaluation_date.year - joining_date.year) * 12 + (evaluation_date.month - joining_date.month)

        if joining_date.day > 10 and joining_date.year == evaluation_date.year and joining_date.month <= evaluation_date.month:
             months_diff -= 1
        elif joining_date.day > 10 and joining_date.year < evaluation_date.year:
             months_diff -= 1


        if months_diff <= 3:
            return "0-3M"
        elif months_diff <= 6:
            return "4-6M"
        else:
            return "BAU"


    def is_month_evaluated(self, joining_date: datetime, evaluation_date: datetime) -> bool:
        """
        Check if a month should be evaluated based on joining date
        All months are evaluated, but joining month after 10th gets special treatment
        """
        is_joining_month_evaluated = (joining_date.month + 1 == evaluation_date.month) and (joining_date.day > 10)

        return is_joining_month_evaluated


    def find_matrix_value(self, matrix: Dict, disbursals: int, logins: int) -> int:
        """
        Find value from effort matrix based on disbursals and logins

        CORRECTED Matrix structure interpretation:
        - First level key (a, b): LOGIN range (not disbursal!)
        - Second level key (c, d): DISBURSAL range (not login!)
        - Value: payout amount

        The axes were swapped in previous implementation!
        Example: (3, inf): {(1, 1): 3500} means:
        - 3+ logins AND exactly 1 disbursal = ₹3500
        """
        # Find the LOGIN range first (outer key)
        login_key = None
        for key in matrix.keys():
            if key[0] <= logins <= key[1]:
                login_key = key
                break

        if login_key is None:
            return 0

        # Find the DISBURSAL range within the login range (inner key)
        disbursal_matrix = matrix[login_key]
        for key in disbursal_matrix.keys():
            if key[0] <= disbursals <= key[1]:
                return disbursal_matrix[key]

        return 0


    def calculate_slab_bonus(self, volume: float, salary: float, slabs: List[Dict]) -> Tuple[float, List[Dict]]:
        """
        Calculate bonus using slab system (like income tax)
        Returns total bonus and breakdown by slab
        """
        total_bonus = 0
        breakdown = []

        for i, slab in enumerate(slabs):
            current_threshold = slab['threshold_multiplier'] * salary

            if i == len(slabs) - 1:  # Last slab
                if volume > current_threshold:
                    taxable_amount = volume - current_threshold
                    slab_bonus = taxable_amount * slab['bonus_rate']
                    total_bonus += slab_bonus
                    breakdown.append({
                        'slab': f"Above {slab['threshold_multiplier']}x salary",
                        'threshold': current_threshold,
                        'rate': slab['bonus_rate'],
                        'taxable_amount': taxable_amount,
                        'bonus': slab_bonus
                    })
            else:
                next_threshold = slabs[i + 1]['threshold_multiplier'] * salary
                if volume > current_threshold:
                    taxable_amount = min(volume, next_threshold) - current_threshold
                    if taxable_amount > 0:
                        slab_bonus = taxable_amount * slab['bonus_rate']
                        total_bonus += slab_bonus
                        breakdown.append({
                            'slab': f"{slab['threshold_multiplier']}x to {slabs[i + 1]['threshold_multiplier']}x salary",
                            'threshold': current_threshold,
                            'rate': slab['bonus_rate'],
                            'taxable_amount': taxable_amount,
                            'bonus': slab_bonus
                        })

        return total_bonus, breakdown

    def calculate_monthly_bonus(self, rm_data: Dict) -> Dict:
        """
        Calculate complete monthly bonus for an RM

        Args:
            rm_data: Dictionary containing RM's monthly performance data

        Returns:
            Dictionary with detailed bonus breakdown
        """

        # Parse dates
        joining_date = datetime.strptime(rm_data['joining_date'], '%d/%m/%Y')
        evaluation_date = datetime.strptime(rm_data['evaluation_date'], '%d/%m/%Y')

        # Calculate evaluation period dynamically
        evaluation_period = self.calculate_evaluation_period(joining_date)
        self.EVALUATION_PERIOD_MONTHS = evaluation_period
        self.update_milestone_qualifying_months(evaluation_period)

        # Check for special case: joining month after 10th
        is_joining_after_10th = self.is_joining_month_after_10th(joining_date, evaluation_date)

        is_joining_month_evaluated = (joining_date.month+1 == evaluation_date.month) and (joining_date.day > 10)

        # Determine vintage category - Pass the is_joining_after_10th flag
        vintage_category = self.get_vintage_category(joining_date, evaluation_date, is_joining_after_10th)

        # Select appropriate slabs and matrices based on vintage
        if vintage_category == "0-3M":
            volume_slabs = self.VINTAGE_VOLUME_SLABS_0_3M
            secured_effort_matrix = self.VINTAGE_SECURED_EFFORT_MATRIX_0_3M
            unsecured_effort_matrix = self.VINTAGE_UNSECURED_EFFORT_MATRIX_0_3M
        elif vintage_category == "4-6M":
            volume_slabs = self.VINTAGE_VOLUME_SLABS_4_6M
            secured_effort_matrix = self.VINTAGE_SECURED_EFFORT_MATRIX_4_6M
            unsecured_effort_matrix = self.VINTAGE_UNSECURED_EFFORT_MATRIX_4_6M
        else:  # BAU
            volume_slabs = self.VOLUME_PAYOUT_SLABS
            secured_effort_matrix = self.BAU_SECURED_EFFORT_MATRIX
            unsecured_effort_matrix = self.BAU_UNSECURED_EFFORT_MATRIX

        # Calculate Volume Bonus
        secured_bonus, secured_breakdown = self.calculate_slab_bonus(
            rm_data['secured_volume'], rm_data['base_salary'], volume_slabs['secured']
        )

        secured_direct_bonus, secured_direct_breakdown = self.calculate_slab_bonus(
            rm_data['secured_direct_volume'], rm_data['base_salary'], volume_slabs['secured_direct']
        )

        unsecured_bonus, unsecured_breakdown = self.calculate_slab_bonus(
            rm_data['unsecured_volume'], rm_data['base_salary'], volume_slabs['unsecured']
        )

        total_volume_bonus = secured_bonus + secured_direct_bonus + unsecured_bonus

        # Calculate Effort Bonus (using corrected matrix access)
        secured_effort_bonus = self.find_matrix_value(
            secured_effort_matrix,
            rm_data['secured_disbursals'],
            rm_data['secured_logins']
        )

        unsecured_effort_bonus = self.find_matrix_value(
            unsecured_effort_matrix,
            rm_data['unsecured_disbursals'],
            rm_data['unsecured_logins']
        )

        total_effort_bonus = secured_effort_bonus + unsecured_effort_bonus

        # Total Monthly Bonus before prorated calculation
        calculated_total_bonus = total_volume_bonus + total_effort_bonus

        # Special handling for joining after 10th of the month in the joining month
        prorated_incentive_amount = 0.0
        special_treatment = "Normal calculation"

        # Check if evaluation month is the month after joining month and joining was after the 10th
        if (evaluation_date.year == joining_date.year and evaluation_date.month == joining_date.month + 1) or \
           (evaluation_date.year == joining_date.year + 1 and joining_date.month == 12 and evaluation_date.month == 1):
            if joining_date.day > 10:
                # Calculate pro-rated ₹5000 based on days worked
                # FIXED: Use joining_date.month instead of undefined 'month'
                month = joining_date.month

                # Get the number of days in the month
                if month in [4, 6, 9, 11]:
                    days_in_month = 30
                elif month in [1, 3, 5, 7, 8, 10, 12]:
                    days_in_month = 31
                elif month == 2:
                    # Leap year check
                    year = joining_date.year
                    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                        days_in_month = 29
                    else:
                        days_in_month = 28
                else:
                    days_in_month = 30

                days_worked = days_in_month - joining_date.day
                prorated_incentive_amount = (5000 * days_worked) / days_in_month

                # Use higher of pro-rated ₹5000 or calculated payout for total bonus
                total_bonus = max(prorated_incentive_amount, calculated_total_bonus)
                special_treatment = f"Joined after 10th in previous month: Used max(₹{prorated_incentive_amount:,.2f} pro-rated, ₹{calculated_total_bonus:,.2f} calculated)"
            else:
                 total_bonus = calculated_total_bonus # Normal calculation if joined before or on 10th
        else:
             total_bonus = calculated_total_bonus # Normal calculation if not the month after joining or joined before or on 10th


        # Calculate milestone for the current month
        milestone_result = self.calculate_milestone_bonus(rm_data)
        milestone_achieved_this_month = milestone_result.get('milestone_achieved_this_month')
        secured_files = milestone_result.get('secured_files_this_month', 0)
        total_volume = milestone_result.get('total_volume_this_month', 0)


        return {
            'total_bonus': total_bonus, # This will be total_monthly_incentive in output
            'evaluation_status': is_joining_month_evaluated,
            'vintage_category': vintage_category,
            'evaluation_period_months': evaluation_period,
            'volume_bonus': total_volume_bonus, # This will be volume_incentive in output
            'effort_bonus': total_effort_bonus, # This will be effort_incentive in output
            'special_treatment': special_treatment,
            'milestone_achieved_this_month': milestone_achieved_this_month,
            'secured_files_this_month': secured_files,
            'total_volume_this_month': total_volume,
            'prorated_incentive_amount': prorated_incentive_amount, # Add prorated amount to result
            'breakdown': {
                'volume_breakdown': {
                    'secured': {'bonus': secured_bonus, 'details': secured_breakdown},
                    'secured_direct': {'bonus': secured_direct_bonus, 'details': secured_direct_breakdown},
                    'unsecured': {'bonus': unsecured_bonus, 'details': unsecured_breakdown}
                },
                'effort_breakdown': {
                    'secured_effort': secured_effort_bonus,
                    'unsecured_effort': unsecured_effort_bonus
                }
            }
        }

    def calculate_milestone_bonus(self, rm_data: Dict) -> Dict:
        """
        Calculate milestone achievement for a single month

        Args:
            rm_data: Dictionary containing RM's monthly performance data

        Returns:
            Dictionary indicating the highest milestone achieved for the month
        """

        if not rm_data:
            return {'milestone_achieved_this_month': None}

        secured_files = rm_data.get('secured_disbursals', 0)
        total_volume = rm_data.get('secured_volume', 0) + rm_data.get('unsecured_volume', 0)

        milestone_achieved_this_month = None

        if (secured_files >= self.MILESTONE_THRESHOLDS['platinum']['min_secured_files'] and
            total_volume >= self.MILESTONE_THRESHOLDS['platinum']['min_volume']):
            milestone_achieved_this_month = 'platinum'
        elif (secured_files >= self.MILESTONE_THRESHOLDS['gold']['min_secured_files'] and
              total_volume >= self.MILESTONE_THRESHOLDS['gold']['min_volume']):
            milestone_achieved_this_month = 'gold'
        elif (secured_files >= self.MILESTONE_THRESHOLDS['silver']['min_secured_files'] and
              total_volume >= self.MILESTONE_THRESHOLDS['silver']['min_volume']):
            milestone_achieved_this_month = 'silver'

        return {
            'milestone_achieved_this_month': milestone_achieved_this_month,
            'secured_files_this_month': secured_files,
            'total_volume_this_month': total_volume
        }
    # _______________________________Redundant Code
    # Keep the original calculate_milestone_bonus as calculate_milestone_rewards to avoid breaking existing code
    def calculate_milestone_rewards(self, rm_monthly_data: List[Dict]) -> Dict:
        """
        Calculate milestone bonus based on sustained performance over evaluation period

        Args:
            rm_monthly_data: List of monthly performance data for evaluation period

        Returns:
            Dictionary with milestone bonus details
        """

        if not rm_monthly_data:
            return {'milestone_achieved': None, 'rewards': {}}

        # Analyze performance across months
        qualifying_months = {'silver': 0, 'gold': 0, 'platinum': 0}

        for month_data in rm_monthly_data:
            secured_files = month_data.get('secured_disbursals', 0)
            total_volume = month_data.get('secured_volume', 0) + month_data.get('unsecured_volume', 0)

            # Check each milestone
            if (secured_files >= self.MILESTONE_THRESHOLDS['platinum']['min_secured_files'] and
                total_volume >= self.MILESTONE_THRESHOLDS['platinum']['min_volume']):
                qualifying_months['platinum'] += 1
            elif (secured_files >= self.MILESTONE_THRESHOLDS['gold']['min_secured_files'] and
                  total_volume >= self.MILESTONE_THRESHOLDS['gold']['min_volume']):
                qualifying_months['gold'] += 1
            elif (secured_files >= self.MILESTONE_THRESHOLDS['silver']['min_secured_files'] and
                  total_volume >= self.MILESTONE_THRESHOLDS['silver']['min_volume']):
                qualifying_months['silver'] += 1

        # Determine highest milestone achieved
        milestone_achieved = None
        base_salary = rm_monthly_data[0].get('base_salary', 0)

        # Assuming all months in rm_monthly_data are within the same evaluation period
        # and the evaluation period is calculated based on the joining date of the first entry
        # This part might need adjustment if rm_monthly_data spans across different evaluation periods
        joining_date_first_month = datetime.strptime(rm_monthly_data[0]['joining_date'], '%d/%m/%Y')
        evaluation_period = self.calculate_evaluation_period(joining_date_first_month)
        self.update_milestone_qualifying_months(evaluation_period)


        if qualifying_months['platinum'] >= self.MILESTONE_QUALIFYING_MONTHS['platinum']:
            milestone_achieved = 'platinum'
        elif qualifying_months['gold'] >= self.MILESTONE_QUALIFYING_MONTHS['gold']:
            milestone_achieved = 'gold'
        elif qualifying_months['silver'] >= self.MILESTONE_QUALIFYING_MONTHS['silver']:
            milestone_achieved = 'silver'

        # Calculate rewards
        rewards = {}
        if milestone_achieved:
            salary_category = 'below_25k' if base_salary < 25000 else 'above_25k'
            salary_hike_percentage = self.MILESTONE_SALARY_HIKE[milestone_achieved][salary_category]
            salary_hike_amount = base_salary * salary_hike_percentage
            saturday_offs = self.MILESTONE_SATURDAY_OFF[milestone_achieved]

            rewards = {
                'milestone': milestone_achieved,
                'salary_hike_percentage': salary_hike_percentage * 100,
                'salary_hike_amount': salary_hike_amount,
                'saturday_offs_per_month': saturday_offs,
                'qualifying_months': qualifying_months.get(milestone_achieved, 0), # Use .get() for safety
                'required_months': self.MILESTONE_QUALIFYING_MONTHS.get(milestone_achieved, 0) # Use .get() for safety
            }

        return {
            'milestone_achieved': milestone_achieved,
            'rewards': rewards,
            'qualifying_months': qualifying_months
        }

## File reading

In [54]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

def process_rm_bonus_csv(input_csv_path, output_csv_path=None):
    # Initialize calculator
    calculator = RMBonusCalculator()

    evaluation_date_str = input("Please enter the evaluation date (DD-MM-YYYY): ")
    print("📁 Reading CSV file...")
    try:
        # Read input CSV
        df_input = pd.read_csv(input_csv_path)
        # print(f"✅ Successfully loaded {len(df_input)} records")

        # Display sample of input data
        # print(f"\n📊 Input Data Sample:")
        # print(df_input.head())

        # Validate required columns
        required_columns = [
            'rm_id', 'sourcing_rm_name', 'joining_date', 'base_salary',
            'secured_logins', 'secured_disbursals', 'unsecured_logins', 'unsecured_disbursals',
            'secured_volume', 'secured_direct_volume', 'unsecured_volume', 'state_name', 'branch_name', 'branch_code'

        ]

        missing_columns = [col for col in required_columns if col not in df_input.columns]
        if missing_columns:
            print(f"❌ Missing required columns: {missing_columns}")
            return None

    except Exception as e:
        print(f"❌ Error reading CSV: {e}")
        return None

    # print(f"\n🔄 Processing {len(df_input)} RM records...")

    # Process each record
    results = []
    errors = []

    date_format_input = '%d/%m/%Y' # Changed from %d-%m-%Y to %d/%m/%y
    date_format_output = '%d/%m/%Y' # Keep output format as DD/MM/YYYY

    try: evaluation_date = datetime.strptime(evaluation_date_str, date_format_input).strftime(date_format_output)
    except ValueError:
        print(f"❌ Invalid date format: {evaluation_date_str}, Please enter in {date_format_input}")
        return None

    for idx, row in df_input.iterrows():
        try:
            # Prepare data for calculator
            rm_data = {
                # Use the correct date format based on the input data
                'joining_date': datetime.strptime(row['joining_date'], date_format_input).strftime(date_format_output),
                'evaluation_date': evaluation_date,
                'base_salary': float(row['base_salary']),
                'secured_logins': int(row['secured_logins']),
                'secured_disbursals': int(row['secured_disbursals']),
                'unsecured_logins': int(row['unsecured_logins']),
                'unsecured_disbursals': int(row['unsecured_disbursals']),
                'secured_volume': float(row['secured_volume']),
                'secured_direct_volume': float(row['secured_direct_volume']),
                'unsecured_volume': float(row['unsecured_volume'])
            }

            # Calculate bonus
            result = calculator.calculate_monthly_bonus(rm_data)

            # Handle potential None values before rounding
            volume_bonus = result.get('volume_bonus', 0.0)
            effort_bonus = result.get('effort_bonus', 0.0)
            total_bonus = result.get('total_bonus', 0.0)
            prorated_incentive_amount = result.get('prorated_incentive_amount', 0.0)


            breakdown = result.get('breakdown', {})
            effort_breakdown = breakdown.get('effort_breakdown', {})
            volume_breakdown = breakdown.get('volume_breakdown', {})

            secured_effort_bonus = effort_breakdown.get('secured_effort', 0.0)
            unsecured_effort_bonus = effort_breakdown.get('unsecured_effort', 0.0)

            secured_volume_bonus = volume_breakdown.get('secured', {}).get('bonus', 0.0)
            secured_direct_bonus = volume_breakdown.get('secured_direct', {}).get('bonus', 0.0)
            unsecured_volume_bonus = volume_breakdown.get('unsecured', {}).get('bonus', 0.0)

            milestone_achieved_this_month = result.get('milestone_achieved_this_month', 'None')
            secured_files = result.get('secured_files_this_month', 0)
            total_volume_this_month = result.get('total_volume_this_month', 0.0)


            # Prepare output record
            output_record = {
                'state_name': row['state_name'],
                'branch_name': row['branch_name'],
                'branch_code': row['branch_code'],

                'rm_id': row['rm_id'],
                'sourcing_rm_name': row['sourcing_rm_name'],
                'joining_date': rm_data['joining_date'], # Use the parsed date
                # 'evaluation_date': evaluation_date, # Use the parsed date
                'base_salary': row['base_salary'],
                'vintage_category': result.get('vintage_category', 'N/A'), # Use .get with default
                'evaluation_period_months': result.get('evaluation_period_months', 'N/A'), # Use .get with default
                'prorated_calculation': result.get('evaluation_status', False), # Use .get with default
                'secured_logins': row['secured_logins'],
                'unsecured_logins': row['unsecured_logins'],
                'secured_disbursals': row['secured_disbursals'],
                'unsecured_disbursals': row['unsecured_disbursals'],
                'secured_volume': row['secured_volume'],
                'secured_direct_volume': row['secured_direct_volume'],
                'unsecured_volume': row['unsecured_volume'],
                'prorated_incentive': round(prorated_incentive_amount, 2), # Added new column
                'volume_incentive': round(volume_bonus, 2), # Renamed
                'effort_incentive': round(effort_bonus, 2), # Renamed
                'secured_effort_incentive': round(secured_effort_bonus, 2), # Renamed
                'unsecured_effort_incentive': round(unsecured_effort_bonus, 2), # Renamed
                'secured_volume_incentive': round(secured_volume_bonus, 2), # Renamed
                'secured_direct_incentive': round(secured_direct_bonus, 2), # Renamed
                'unsecured_volume_incentive': round(unsecured_volume_bonus, 2), # Renamed
                'total_monthly_incentive': round(total_bonus, 2), # Renamed
                'milestone_achieved': milestone_achieved_this_month,
                'secured_files': secured_files,
                'total_volume': round(total_volume_this_month, 2)

            }


            results.append(output_record)

            # Progress indicator
            # if (idx + 1) % 10 == 0:
            #     print(f"  ✅ Processed {idx + 1}/{len(df_input)} records")

        except Exception as e:
            error_record = {
                'rm_id': row.get('rm_id', f'Row_{idx}'),
                'error': str(e)
            }
            errors.append(error_record)
            print(f"  ❌ Error processing row {idx + 1}: {e}")

    # Create output DataFrame
    df_output = pd.DataFrame(results)

    # Replace None/NaN in 'milestone_achieved_this_month' with '-'
    if 'milestone_achieved_this_month' in df_output.columns:
        df_output['milestone_achieved_this_month'].fillna('-', inplace=True)
        df_output['milestone_achieved_this_month'] = df_output['milestone_achieved_this_month'].replace('', '-')

    print(f"\n📊 Processing Complete!")
    print(f"✅ Successfully processed: {len(results)} records")
    print(f"❌ Errors encountered: {len(errors)} records")

    if errors:
        print(f"\n⚠️ Error Details:")
        for error in errors:
            print(f"  • RM ID {error['rm_id']}: {error['error']}")

    # Generate output file path if not provided
    if output_csv_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_csv_path = f"rm_bonus_results_{timestamp}.csv" # Keep bonus in filename

    # Save to CSV
    try:
        if not df_output.empty:
            df_output.to_csv(output_csv_path, index=False)
            print(f"\n💾 Results saved to: {output_csv_path}")

            # Display sample output
            # print(f"\n📈 Output Sample:")
            # print(df_output[['rm_id', 'rm_name', 'vintage_category', 'total_monthly_bonus', 'volume_bonus', 'effort_bonus', 'prorated_calculation']].head())

            # Summary statistics
            print(f"\n📊 Summary Statistics:")
            print(f"Average Total Incentive: ₹{df_output['total_monthly_incentive'].mean():,.2f}") # Renamed
            print(f"Maximum Total Incentive: ₹{df_output['total_monthly_incentive'].max():,.2f}") # Renamed
            print(f"Minimum Total Incentive: ₹{df_output['total_monthly_incentive'].min():,.2f}") # Renamed
            print(f"Total Incentive Payout: ₹{df_output['total_monthly_incentive'].sum():,.2f}") # Renamed


            # Vintage distribution
            print(f"\n📈 Vintage Distribution:")
            vintage_counts = df_output['vintage_category'].value_counts()
            for vintage, count in vintage_counts.items():
                print(f"  {vintage}: {count} RMs")

            # Milestone distribution
            print(f"\n🤩 Milestone Distribution:")
            milestone_counts = df_output['milestone_achieved_this_month'].value_counts()
            for milestone, count in milestone_counts.items():
                print(f"  {milestone}: {count} RMs")

            # Prorated Calculation
            print(f"\n😶‍🌫️ Prorated Calculation:")
            prorated_counts = df_output['prorated_calculation'].value_counts()
            for prorated, count in prorated_counts.items():
                print(f"   {prorated} : {count} RMs")
        else:
            print("\n⚠️ No successful records to save or display.")


    except Exception as e:
        print(f"❌ Error saving CSV: {e}")
        return df_output

    return df_output

## Final Output

In [55]:
process_rm_bonus_csv('/content/RM_input_data.csv')

Please enter the evaluation date (DD-MM-YYYY): 01/10/2025
📁 Reading CSV file...
  ❌ Error processing row 269: strptime() argument 1 must be str, not float
  ❌ Error processing row 270: strptime() argument 1 must be str, not float
  ❌ Error processing row 297: strptime() argument 1 must be str, not float

📊 Processing Complete!
✅ Successfully processed: 344 records
❌ Errors encountered: 3 records

⚠️ Error Details:
  • RM ID ygm0003: strptime() argument 1 must be str, not float
  • RM ID ygm0001: strptime() argument 1 must be str, not float
  • RM ID sf0209: strptime() argument 1 must be str, not float

💾 Results saved to: rm_bonus_results_20251007_110504.csv

📊 Summary Statistics:
Average Total Incentive: ₹9,693.87
Maximum Total Incentive: ₹47,004.38
Minimum Total Incentive: ₹1,200.00
Total Incentive Payout: ₹3,334,690.75

📈 Vintage Distribution:
  0-3M: 196 RMs
  4-6M: 143 RMs
  BAU: 5 RMs

🤩 Milestone Distribution:
  -: 228 RMs
  silver: 55 RMs
  gold: 44 RMs
  platinum: 17 RMs

😶‍🌫️

,state_name,branch_name,branch_code,rm_id,sourcing_rm_name,joining_date,evaluation_date,base_salary,vintage_category,prorated_calculation,...,secured_effort_incentive,unsecured_effort_incentive,secured_volume_incentive,secured_direct_incentive,unsecured_volume_incentive,total_monthly_incentive,milestone_achieved_this_month,secured_files,total_volume,prorated_incentive_amount
0,AP,Bhimavaram,AP07,sf0477,Mr Anil Kumar,11/07/2025,01/10/2025,32500.0,0-3M,False,...,3000,0,343.75,360.00,0.0,3703.75,-,0,300000.0,0.0
1,AP,Bhimavaram,AP07,sf0107,Mr Challa SivaSankara,09/04/2025,01/10/2025,31250.0,4-6M,False,...,3500,0,10356.25,7737.50,0.0,21593.75,silver,2,3100000.0,0.0
2,AP,Bhimavaram,AP07,sf0636,Mr Chevveti Vijay Kumar,09/09/2025,01/10/2025,20833.0,0-3M,False,...,3500,0,1422.93,947.93,0.0,5870.85,-,1,600000.0,0.0
3,AP,Bhimavaram,AP07,sf0096,Mr Pecheti VaraPrasad,05/04/2025,01/10/2025,31250.0,4-6M,False,...,2000,0,375.00,593.75,0.0,2968.75,-,0,500000.0,0.0
4,AP,Bhimavaram,AP07,sf0298,Mr Veera Revathi,02/06/2025,01/10/2025,29167.0,4-6M,False,...,2500,0,2652.14,2390.16,0.0,7542.30,-,1,1282835.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,UP,Sitapur,UP03,sf0274,Mr Amresh Pal,02/06/2025,01/10/2025,15000.0,4-6M,False,...,1200,0,0.00,0.00,0.0,1200.00,-,0,0.0,0.0
340,UP,Sitapur,UP03,sf0283,Mr Kartikey Singh,02/06/2025,01/10/2025,16667.0,4-6M,False,...,1200,0,0.00,0.00,0.0,1200.00,-,0,0.0,0.0
341,UP,Sitapur,UP03,sf0347,Mr Prabhat Singh,12/06/2025,01/10/2025,16667.0,0-3M,False,...,3000,0,0.00,0.00,0.0,3000.00,-,0,0.0,0.0
342,UP,Varanasi,UP09,sf0652,Mr Shailesh Singh,15/09/2025,01/10/2025,26667.0,0-3M,True,...,2500,0,0.00,0.00,0.0,2500.00,-,0,0.0,2500.0


## One on one

In [ ]:
# Test the system with provided data
def test_bonus_calculator():
    """Test the bonus calculator with provided sample data"""

    calculator = RMBonusCalculator()

    Test_Case_1 = "Nagaraju 10L direct case"
    rm_data_1 = {
        'joining_date': '30/06/2025',
        'evaluation_date': '30/06/2025',
        'base_salary': 33750,
        'secured_logins': 0,
        'secured_disbursals': 0,
        'unsecured_logins': 0,
        'unsecured_disbursals': 0,
        'secured_volume': 0,
        'secured_direct_volume': 0,
        'unsecured_volume': 0
    }

    # Add a test case for joining after the 10th to check vintage
    Test_Case_2 = "RM joined April 14th, evaluated August 1st"
    rm_data_2 = {
        'joining_date': '14/04/2025',
        'evaluation_date': '01/08/2025',
        'base_salary': 28000,
        'secured_logins': 5,
        'secured_disbursals': 2,
        'unsecured_logins': 3,
        'unsecured_disbursals': 1,
        'secured_volume': 1000000,
        'secured_direct_volume': 500000,
        'unsecured_volume': 300000
    }


    print("🏦 RM Bonus Calculator - UPDATED VERSION")
    print("=" * 60)

    # Test both cases
    for i, rm_data in enumerate([rm_data_1, rm_data_2], 1):
        print(f"\n{'='*20} TEST_CASE_{i} {'='*20}")

        # Calculate monthly bonus
        result = calculator.calculate_monthly_bonus(rm_data)

        print(f"\n📊 RM Performance Summary:")
        print(f"Joining Date: {rm_data['joining_date']}")
        print(f"Evaluation Date: {rm_data['evaluation_date']}")
        print(f"Base Salary: ₹{rm_data['base_salary']:,}")
        print(f"Vintage Category: {result['vintage_category']}")
        print(f"Evaluation Period: {result.get('evaluation_period_months', 'N/A')} months")
        print(f"Evaluation Status (Joining Month Evaluated): {result['evaluation_status']}")
        print(f"Special Treatment: {result['special_treatment']}")


        print(f"\n💰 Monthly Bonus Breakdown:")
        print(f"Volume Bonus: ₹{result['volume_bonus']:,.2f}")
        print(f"Effort Bonus: ₹{result['effort_bonus']:,.2f}")
        print(f"Total Monthly Bonus: ₹{result['total_bonus']:,.2f}")

                # Volume breakdown
        print(f"\n📈 Volume Bonus Details:")
        vol_breakdown = result['breakdown']['volume_breakdown']

        print(f"  Secured Volume Bonus: ₹{vol_breakdown['secured']['bonus']:,.2f}")
        for detail in vol_breakdown['secured']['details']:
            print(f"    • {detail['slab']}: ₹{detail['taxable_amount']:,.0f} @ {detail['rate']*100:.2f}% = ₹{detail['bonus']:,.2f}")

        print(f"  Secured Direct Bonus: ₹{vol_breakdown['secured_direct']['bonus']:,.2f}")
        for detail in vol_breakdown['secured_direct']['details']:
            print(f"    • {detail['slab']}: ₹{detail['taxable_amount']:,.0f} @ {detail['rate']*100:.2f}% = ₹{detail['bonus']:,.2f}")

        print(f"  Unsecured Volume Bonus: ₹{vol_breakdown['unsecured']['bonus']:,.2f}")

        # Effort breakdown with matrix explanation
        print(f"\n⚡ Effort Bonus Matrix Analysis:")
        effort_breakdown = result['breakdown']['effort_breakdown']
        print(f"  Secured Effort: {rm_data['secured_logins']} logins, {rm_data['secured_disbursals']} disbursals → ₹{effort_breakdown['secured_effort']:,.2f}")
        print(f"  Unsecured Effort: {rm_data['unsecured_logins']} logins, {rm_data['unsecured_disbursals']} disbursals → ₹{effort_breakdown['unsecured_effort']:,.2f}")


                # Effort breakdown with detailed explanation
        print(f"\n⚡ Effort Bonus Details:")
        effort_breakdown = result['breakdown']['effort_breakdown']
        print(f"  Secured Effort Bonus: ₹{effort_breakdown['secured_effort']:,.2f}")
        print(f"    (Based on {rm_data['secured_disbursals']} disbursals, {rm_data['secured_logins']} logins)")
        print(f"  Unsecured Effort Bonus: ₹{effort_breakdown['unsecured_effort']:,.2f}")
        print(f"    (Based on {rm_data['unsecured_disbursals']} disbursals, {rm_data['unsecured_logins']} logins)")
        print(f"    📝 Matrix Logic: 0 disbursals → use (0,0) entry regardless of logins = ₹0")

    return result

# Additional utility functions for comprehensive analysis
def create_performance_dashboard(monthly_data_list: List[Dict]) -> pd.DataFrame:
    """Create a performance dashboard from multiple months of data"""

    calculator = RMBonusCalculator()
    dashboard_data = []

    for month_data in monthly_data_list:
        result = calculator.calculate_monthly_bonus(month_data)

        dashboard_data.append({
            'Month': month_data['evaluation_date'],
            'Vintage': result['vintage_category'],
            'Volume_Bonus': result['volume_bonus'],
            'Effort_Bonus': result['effort_bonus'],
            'Total_Bonus': result['total_bonus'],
            'Secured_Volume': month_data['secured_volume'],
            'Unsecured_Volume': month_data['unsecured_volume'],
            'Secured_Disbursals': month_data['secured_disbursals'],
            'Secured_Logins': month_data['secured_logins']
        })

    return pd.DataFrame(dashboard_data)

def analyze_milestone_progression(rm_monthly_data: List[Dict]) -> Dict:
    """Analyze milestone progression over time"""

    calculator = RMBonusCalculator()
    milestone_result = calculator.calculate_milestone_bonus(rm_monthly_data)

    # Add month-by-month analysis
    monthly_milestone_analysis = []
    for month_data in rm_monthly_data:
        secured_files = month_data.get('secured_disbursals', 0)
        total_volume = month_data.get('secured_volume', 0) + month_data.get('unsecured_volume', 0)

        monthly_milestone_analysis.append({
            'month': month_data['evaluation_date'],
            'secured_files': secured_files,
            'total_volume': total_volume,
            'meets_silver': (secured_files >= 2 and total_volume >= 1500000),
            'meets_gold': (secured_files >= 3 and total_volume >= 2000000),
            'meets_platinum': (secured_files >= 4 and total_volume >= 3000000)
        })

    milestone_result['monthly_analysis'] = monthly_milestone_analysis
    return milestone_result

# Run the test
if __name__ == "__main__":
    test_result = test_bonus_calculator()

print("\n🚀 RM Bonus Calculator System Ready!")
print("Use the RMBonusCalculator class to calculate bonuses for any RM.")
print("Key methods:")
print("• calculate_monthly_bonus(rm_data) - Calculate monthly bonus")
print("• calculate_milestone_bonus(monthly_data_list) - Calculate milestone rewards")
print("• create_performance_dashboard(monthly_data_list) - Create performance dashboard")
print("• calculate_evaluation_period(joining_date) - Get evaluation period based on financial quarter")

🏦 RM Bonus Calculator - UPDATED VERSION

==================== TEST_CASE_1 ====================

📊 RM Performance Summary:
Joining Date: 30/06/2025
Evaluation Date: 30/06/2025
Base Salary: ₹33,750
Vintage Category: 0-3M
Evaluation Period: 5 months
Evaluation Status: Evaluated

💰 Monthly Bonus Breakdown:
Volume Bonus: ₹0.00
Effort Bonus: ₹1,500.00
Total Monthly Bonus: ₹1,500.00

📈 Volume Bonus Details:
  Secured Volume Bonus: ₹0.00
  Secured Direct Bonus: ₹0.00
  Unsecured Volume Bonus: ₹0.00

⚡ Effort Bonus Matrix Analysis:
  Secured Effort: 0 logins, 0 disbursals → ₹1,500.00
  Unsecured Effort: 0 logins, 0 disbursals → ₹0.00

⚡ Effort Bonus Details:
  Secured Effort Bonus: ₹1,500.00
    (Based on 0 disbursals, 0 logins)
  Unsecured Effort Bonus: ₹0.00
    (Based on 0 disbursals, 0 logins)
    📝 Matrix Logic: 0 disbursals → use (0,0) entry regardless of logins = ₹0

🚀 RM Bonus Calculator System Ready!
Use the RMBonusCalculator class to calculate bonuses for any RM.
Key methods:
• calcul